# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*NEGATIVE_SAMPLE, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

# 定义模型

In [6]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        #return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        #return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')

    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [7]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.2   # 原来是4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
NEGATIVE_SAMPLE = 6
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)
    
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 5444.7632
Epoch: 01, Loss: 5385.4072
Epoch: 02, Loss: 5277.6064
Epoch: 03, Loss: 5142.3047
Epoch: 04, Loss: 4998.9229
AP SCORE：  0.5582299717810106
AUC SCORE:  0.5197704081632654
Epoch: 00, Loss: 4850.6646
Epoch: 01, Loss: 4847.5469
Epoch: 02, Loss: 4842.4326
Epoch: 03, Loss: 4836.7744
Epoch: 04, Loss: 4831.7437
AP SCORE：  0.548024907762138
AUC SCORE:  0.5379464285714286
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4827.6631
Epoch: 01, Loss: 4791.5259
Epoch: 02, Loss: 4731.5918
Epoch: 03, Loss: 4656.2402
Epoch: 04, Loss: 4570.2734
AP SCORE：  0.5644608056470998
AUC SCORE:  0.5704719387755102
Epoch: 00, Loss: 4476.6343
Epoch: 01, Loss: 4471.5601
Epoch: 02, Loss: 4463.5063
Epoch: 03, Loss: 4455.6074
Epoch: 04, Loss: 4450.9688
AP SCORE：  0.6156382828282836
AUC SCORE:  0.6211734693877551
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4449.9043
Epoc

Epoch: 00, Loss: 3129.8254
Epoch: 01, Loss: 3127.3137
Epoch: 02, Loss: 3122.5676
Epoch: 03, Loss: 3115.8574
Epoch: 04, Loss: 3107.4485
AP SCORE：  0.7662752013076252
AUC SCORE:  0.7506377551020409
Epoch: 00, Loss: 3097.5991
Epoch: 01, Loss: 3097.5322
Epoch: 02, Loss: 3097.4258
Epoch: 03, Loss: 3097.3110
Epoch: 04, Loss: 3097.2146
AP SCORE：  0.7683289115809614
AUC SCORE:  0.7522321428571427
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 3097.1438
Epoch: 01, Loss: 3094.7698
Epoch: 02, Loss: 3090.2881
Epoch: 03, Loss: 3083.9629
Epoch: 04, Loss: 3076.0449
AP SCORE：  0.7704839881920168
AUC SCORE:  0.7544642857142857
Epoch: 00, Loss: 3066.7698
Epoch: 01, Loss: 3066.7168
Epoch: 02, Loss: 3066.6316
Epoch: 03, Loss: 3066.5383
Epoch: 04, Loss: 3066.4556
AP SCORE：  0.7719082675831394
AUC SCORE:  0.7557397959183674
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 3066.3894
Epoch: 01, Loss: 3064.1396
Epoch: 02, Loss: 3059.8904
Epoch: 03

Epoch: 01, Loss: 2699.7109
Epoch: 02, Loss: 2697.0769
Epoch: 03, Loss: 2693.3447
Epoch: 04, Loss: 2688.6494
AP SCORE：  0.8019630986904537
AUC SCORE:  0.775829081632653
Epoch: 00, Loss: 2683.1172
Epoch: 01, Loss: 2683.0969
Epoch: 02, Loss: 2683.0667
Epoch: 03, Loss: 2683.0359
Epoch: 04, Loss: 2683.0125
AP SCORE：  0.8020152580038101
AUC SCORE:  0.7758290816326531
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2682.9980
Epoch: 01, Loss: 2681.6487
Epoch: 02, Loss: 2679.0952
Epoch: 03, Loss: 2675.4771
Epoch: 04, Loss: 2670.9241
AP SCORE：  0.8004093699414793
AUC SCORE:  0.7751913265306123
Epoch: 00, Loss: 2665.5608
Epoch: 01, Loss: 2665.5417
Epoch: 02, Loss: 2665.5132
Epoch: 03, Loss: 2665.4854
Epoch: 04, Loss: 2665.4644
AP SCORE：  0.8007212789329656
AUC SCORE:  0.7758290816326531
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2665.4512
Epoch: 01, Loss: 2664.1436
Epoch: 02, Loss: 2661.6680
Epoch: 03, Loss: 2658.1589
Epoch: 04,

Epoch: 02, Loss: 2423.7312
Epoch: 03, Loss: 2421.0837
Epoch: 04, Loss: 2417.7478
AP SCORE：  0.805377073831482
AUC SCORE:  0.7802933673469388
Epoch: 00, Loss: 2413.8091
Epoch: 01, Loss: 2413.7920
Epoch: 02, Loss: 2413.7754
Epoch: 03, Loss: 2413.7627
Epoch: 04, Loss: 2413.7534
AP SCORE：  0.805343231497382
AUC SCORE:  0.7799744897959184
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 2413.7515
Epoch: 01, Loss: 2412.7847
Epoch: 02, Loss: 2410.9541
Epoch: 03, Loss: 2408.3574
Epoch: 04, Loss: 2405.0852
AP SCORE：  0.8069271836557177
AUC SCORE:  0.7809311224489796
Epoch: 00, Loss: 2401.2205
Epoch: 01, Loss: 2401.2048
Epoch: 02, Loss: 2401.1897
Epoch: 03, Loss: 2401.1782
Epoch: 04, Loss: 2401.1694
AP SCORE：  0.8073572085422643
AUC SCORE:  0.7809311224489796
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 2401.1672
Epoch: 01, Loss: 2400.2195
Epoch: 02, Loss: 2398.4238
Epoch: 03, Loss: 2395.8760
Epoch: 04, Loss: 2392.6633
AP SCORE：  

Epoch: 03, Loss: 2216.3608
Epoch: 04, Loss: 2213.7061
AP SCORE：  0.7806905384312973
AUC SCORE:  0.754783163265306
Epoch: 00, Loss: 2210.5696
Epoch: 01, Loss: 2210.5579
Epoch: 02, Loss: 2210.5447
Epoch: 03, Loss: 2210.5352
Epoch: 04, Loss: 2210.5295
AP SCORE：  0.780960987259371
AUC SCORE:  0.7551020408163266
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 2210.5288
Epoch: 01, Loss: 2209.7583
Epoch: 02, Loss: 2208.2979
Epoch: 03, Loss: 2206.2256
Epoch: 04, Loss: 2203.6123
AP SCORE：  0.7784807511145629
AUC SCORE:  0.7554209183673469
Epoch: 00, Loss: 2200.5244
Epoch: 01, Loss: 2200.5129
Epoch: 02, Loss: 2200.5007
Epoch: 03, Loss: 2200.4919
Epoch: 04, Loss: 2200.4866
AP SCORE：  0.7767423907539172
AUC SCORE:  0.7554209183673468
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 2200.4861
Epoch: 01, Loss: 2199.7275
Epoch: 02, Loss: 2198.2903
Epoch: 03, Loss: 2196.2502
Epoch: 04, Loss: 2193.6777
AP SCORE：  0.7748307525476693
AUC SCOR

...

Epoch: 02, Loss: 2976.0408
Epoch: 03, Loss: 2970.6934
Epoch: 04, Loss: 2963.9788
AP SCORE：  0.6358572241074966
AUC SCORE:  0.6705994897959183
Epoch: 00, Loss: 2956.0820
Epoch: 01, Loss: 2956.0371
Epoch: 02, Loss: 2955.9719
Epoch: 03, Loss: 2955.9136
Epoch: 04, Loss: 2955.8748
AP SCORE：  0.6376853504011285
AUC SCORE:  0.6721938775510204
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 2955.8513
Epoch: 01, Loss: 2953.9065
Epoch: 02, Loss: 2950.2317
Epoch: 03, Loss: 2945.0359
Epoch: 04, Loss: 2938.5193
AP SCORE：  0.6390291634201481
AUC SCORE:  0.6728316326530612
Epoch: 00, Loss: 2930.8706
Epoch: 01, Loss: 2930.8259
Epoch: 02, Loss: 2930.7656
Epoch: 03, Loss: 2930.7141
Epoch: 04, Loss: 2930.6775
AP SCORE：  0.6395557240166558
AUC SCORE:  0.6741071428571429
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 2930.6528
Epoch: 01, Loss: 2928.7815
Epoch: 02, Loss: 2925.2480
Epoch: 03, Loss: 2920.2549
Epoch: 04, Loss: 2913.9951
AP SCORE：

Epoch: 03, Loss: 2607.6340
Epoch: 04, Loss: 2603.4668
AP SCORE：  0.6804811320676584
AUC SCORE:  0.6910076530612245
Epoch: 00, Loss: 2598.5435
Epoch: 01, Loss: 2598.5081
Epoch: 02, Loss: 2598.4819
Epoch: 03, Loss: 2598.4561
Epoch: 04, Loss: 2598.4253
AP SCORE：  0.6805328522565093
AUC SCORE:  0.6910076530612245
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 2598.4155
Epoch: 01, Loss: 2597.1975
Epoch: 02, Loss: 2594.8906
Epoch: 03, Loss: 2591.6174
Epoch: 04, Loss: 2587.4915
AP SCORE：  0.6860471408465185
AUC SCORE:  0.6919642857142857
Epoch: 00, Loss: 2582.6191
Epoch: 01, Loss: 2582.5835
Epoch: 02, Loss: 2582.5552
Epoch: 03, Loss: 2582.5281
Epoch: 04, Loss: 2582.4971
AP SCORE：  0.6855875941230538
AUC SCORE:  0.6913265306122449
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 2582.4885
Epoch: 01, Loss: 2581.2834
Epoch: 02, Loss: 2579.0032
Epoch: 03, Loss: 2575.7708
Epoch: 04, Loss: 2571.7012
AP SCORE：  0.6862103819313583
AUC SC

Epoch: 04, Loss: 2353.8865
AP SCORE：  0.7226743276219856
AUC SCORE:  0.7037627551020409
Epoch: 00, Loss: 2350.2673
Epoch: 01, Loss: 2350.2444
Epoch: 02, Loss: 2350.2346
Epoch: 03, Loss: 2350.2229
Epoch: 04, Loss: 2350.2080
AP SCORE：  0.7236006494787605
AUC SCORE:  0.7044005102040817
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 2350.2095
Epoch: 01, Loss: 2349.3196
Epoch: 02, Loss: 2347.6348
Epoch: 03, Loss: 2345.2439
Epoch: 04, Loss: 2342.2288
AP SCORE：  0.7233726513049549
AUC SCORE:  0.7050382653061225
Epoch: 00, Loss: 2338.6641
Epoch: 01, Loss: 2338.6418
Epoch: 02, Loss: 2338.6318
Epoch: 03, Loss: 2338.6204
Epoch: 04, Loss: 2338.6057
AP SCORE：  0.723447799123503
AUC SCORE:  0.7044005102040817
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 2338.6064
Epoch: 01, Loss: 2337.7300
Epoch: 02, Loss: 2336.0698
Epoch: 03, Loss: 2333.7134
Epoch: 04, Loss: 2330.7412
AP SCORE：  0.7240411478937362
AUC SCORE:  0.7050382653061226
Epo

Epoch: 00, Loss: 2159.4551
Epoch: 01, Loss: 2159.4397
Epoch: 02, Loss: 2159.4275
Epoch: 03, Loss: 2159.4155
Epoch: 04, Loss: 2159.4060
AP SCORE：  0.7242634108938165
AUC SCORE:  0.7040816326530611
######################### 模块循环 ： 78 ##########################
Epoch: 00, Loss: 2159.4050
Epoch: 01, Loss: 2158.6724
Epoch: 02, Loss: 2157.2842
Epoch: 03, Loss: 2155.3140
Epoch: 04, Loss: 2152.8286
AP SCORE：  0.7245255487006598
AUC SCORE:  0.7037627551020408
Epoch: 00, Loss: 2149.8899
Epoch: 01, Loss: 2149.8738
Epoch: 02, Loss: 2149.8611
Epoch: 03, Loss: 2149.8489
Epoch: 04, Loss: 2149.8389
AP SCORE：  0.7236176798346795
AUC SCORE:  0.7024872448979592
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 2149.8379
Epoch: 01, Loss: 2149.1121
Epoch: 02, Loss: 2147.7378
Epoch: 03, Loss: 2145.7874
Epoch: 04, Loss: 2143.3284
AP SCORE：  0.7227962829265531
AUC SCORE:  0.701530612244898
Epoch: 00, Loss: 2140.4221
Epoch: 01, Loss: 2140.4055
Epoch: 02, Loss: 2140.3921
Epoch: 03,

Epoch: 01, Loss: 2002.4734
Epoch: 02, Loss: 2002.4663
Epoch: 03, Loss: 2002.4573
Epoch: 04, Loss: 2002.4496
AP SCORE：  0.7094830245786187
AUC SCORE:  0.6852678571428571
######################### 模块循环 ： 96 ##########################
Epoch: 00, Loss: 2002.4509
Epoch: 01, Loss: 2001.8604
Epoch: 02, Loss: 2000.7421
Epoch: 03, Loss: 1999.1549
Epoch: 04, Loss: 1997.1538
AP SCORE：  0.7100809294865033
AUC SCORE:  0.685905612244898
Epoch: 00, Loss: 1994.7887
Epoch: 01, Loss: 1994.7740
Epoch: 02, Loss: 1994.7677
Epoch: 03, Loss: 1994.7593
Epoch: 04, Loss: 1994.7515
AP SCORE：  0.7096173863611718
AUC SCORE:  0.6849489795918366
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 1994.7531
Epoch: 01, Loss: 1994.1711
Epoch: 02, Loss: 1993.0693
Epoch: 03, Loss: 1991.5057
Epoch: 04, Loss: 1989.5339
AP SCORE：  0.7068405722267576
AUC SCORE:  0.6836734693877551
Epoch: 00, Loss: 1987.2039
Epoch: 01, Loss: 1987.1896
Epoch: 02, Loss: 1987.1838
Epoch: 03, Loss: 1987.1755
Epoch: 04,

In [8]:
context_embedding_dim = 12
importance_embedding_dim = 88
beta = 4.2   # 原来是4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
NEGATIVE_SAMPLE = 6
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)
    
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 5417.6133
Epoch: 01, Loss: 5360.8843
Epoch: 02, Loss: 5255.9092
Epoch: 03, Loss: 5122.2949
Epoch: 04, Loss: 4980.8057
AP SCORE：  0.6117341291248848
AUC SCORE:  0.5558035714285715
Epoch: 00, Loss: 4837.0947
Epoch: 01, Loss: 4836.2471
Epoch: 02, Loss: 4834.7192
Epoch: 03, Loss: 4832.7168
Epoch: 04, Loss: 4830.4473
AP SCORE：  0.6266419782790043
AUC SCORE:  0.5809948979591837
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4828.0771
Epoch: 01, Loss: 4793.4834
Epoch: 02, Loss: 4733.9390
Epoch: 03, Loss: 4659.3354
Epoch: 04, Loss: 4574.7095
AP SCORE：  0.6570401172391698
AUC SCORE:  0.6285076530612245
Epoch: 00, Loss: 4481.6953
Epoch: 01, Loss: 4478.6797
Epoch: 02, Loss: 4473.2998
Epoch: 03, Loss: 4467.2720
Epoch: 04, Loss: 4463.3350
AP SCORE：  0.6525505210791396
AUC SCORE:  0.642219387755102
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4460.3662
Epoc

Epoch: 00, Loss: 3135.0161
Epoch: 01, Loss: 3132.6545
Epoch: 02, Loss: 3128.1968
Epoch: 03, Loss: 3121.9014
Epoch: 04, Loss: 3114.0190
AP SCORE：  0.7486175573717845
AUC SCORE:  0.7557397959183673
Epoch: 00, Loss: 3104.7849
Epoch: 01, Loss: 3104.6709
Epoch: 02, Loss: 3104.4678
Epoch: 03, Loss: 3104.2078
Epoch: 04, Loss: 3103.9236
AP SCORE：  0.7498733803013845
AUC SCORE:  0.7573341836734694
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 3103.6401
Epoch: 01, Loss: 3101.3894
Epoch: 02, Loss: 3097.1389
Epoch: 03, Loss: 3091.1348
Epoch: 04, Loss: 3083.6125
AP SCORE：  0.7499860849942432
AUC SCORE:  0.7573341836734694
Epoch: 00, Loss: 3074.7947
Epoch: 01, Loss: 3074.6826
Epoch: 02, Loss: 3074.4839
Epoch: 03, Loss: 3074.2273
Epoch: 04, Loss: 3073.9412
AP SCORE：  0.7510547676609863
AUC SCORE:  0.7589285714285714
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 3073.6482
Epoch: 01, Loss: 3071.4956
Epoch: 02, Loss: 3067.4314
Epoch: 03

Epoch: 01, Loss: 2714.5273
Epoch: 02, Loss: 2711.9524
Epoch: 03, Loss: 2708.3081
Epoch: 04, Loss: 2703.7319
AP SCORE：  0.7643532683253692
AUC SCORE:  0.7809311224489797
Epoch: 00, Loss: 2698.3472
Epoch: 01, Loss: 2698.2998
Epoch: 02, Loss: 2698.2224
Epoch: 03, Loss: 2698.1328
Epoch: 04, Loss: 2698.0479
AP SCORE：  0.7631739889181041
AUC SCORE:  0.7806122448979592
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2697.9766
Epoch: 01, Loss: 2696.6475
Epoch: 02, Loss: 2694.1333
Epoch: 03, Loss: 2690.5745
Epoch: 04, Loss: 2686.1018
AP SCORE：  0.7636642437637174
AUC SCORE:  0.7809311224489796
Epoch: 00, Loss: 2680.8369
Epoch: 01, Loss: 2680.7939
Epoch: 02, Loss: 2680.7231
Epoch: 03, Loss: 2680.6431
Epoch: 04, Loss: 2680.5688
AP SCORE：  0.7637107429319965
AUC SCORE:  0.7809311224489797
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2680.5083
Epoch: 01, Loss: 2679.2080
Epoch: 02, Loss: 2676.7495
Epoch: 03, Loss: 2673.2676
Epoch: 04

Epoch: 02, Loss: 2440.4070
Epoch: 03, Loss: 2437.7021
Epoch: 04, Loss: 2434.2935
AP SCORE：  0.7960210446889461
AUC SCORE:  0.7924107142857143
Epoch: 00, Loss: 2430.2703
Epoch: 01, Loss: 2430.2336
Epoch: 02, Loss: 2430.1738
Epoch: 03, Loss: 2430.1052
Epoch: 04, Loss: 2430.0388
AP SCORE：  0.79583646364499
AUC SCORE:  0.7920918367346939
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 2429.9827
Epoch: 01, Loss: 2428.9814
Epoch: 02, Loss: 2427.0879
Epoch: 03, Loss: 2424.4058
Epoch: 04, Loss: 2421.0322
AP SCORE：  0.7962444370082254
AUC SCORE:  0.7920918367346939
Epoch: 00, Loss: 2417.0579
Epoch: 01, Loss: 2417.0242
Epoch: 02, Loss: 2416.9688
Epoch: 03, Loss: 2416.9067
Epoch: 04, Loss: 2416.8491
AP SCORE：  0.7975723625446373
AUC SCORE:  0.7924107142857142
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 2416.8022
Epoch: 01, Loss: 2415.8152
Epoch: 02, Loss: 2413.9478
Epoch: 03, Loss: 2411.3008
Epoch: 04, Loss: 2407.9690
AP SCORE：  

Epoch: 03, Loss: 2225.7937
Epoch: 04, Loss: 2223.1941
AP SCORE：  0.8043726740615659
AUC SCORE:  0.7822066326530612
Epoch: 00, Loss: 2220.1240
Epoch: 01, Loss: 2220.1008
Epoch: 02, Loss: 2220.0618
Epoch: 03, Loss: 2220.0156
Epoch: 04, Loss: 2219.9688
AP SCORE：  0.804139416752837
AUC SCORE:  0.7818877551020408
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 2219.9246
Epoch: 01, Loss: 2219.1650
Epoch: 02, Loss: 2217.7271
Epoch: 03, Loss: 2215.6868
Epoch: 04, Loss: 2213.1150
AP SCORE：  0.8045594884844223
AUC SCORE:  0.7812499999999999
Epoch: 00, Loss: 2210.0754
Epoch: 01, Loss: 2210.0508
Epoch: 02, Loss: 2210.0093
Epoch: 03, Loss: 2209.9590
Epoch: 04, Loss: 2209.9067
AP SCORE：  0.8044126851743537
AUC SCORE:  0.78125
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 2209.8560
Epoch: 01, Loss: 2209.1030
Epoch: 02, Loss: 2207.6775
Epoch: 03, Loss: 2205.6548
Epoch: 04, Loss: 2203.1040
AP SCORE：  0.8038214881293071
AUC SCORE:  0.7806

...

Epoch: 01, Loss: 2972.7493
Epoch: 02, Loss: 2969.2146
Epoch: 03, Loss: 2964.2183
Epoch: 04, Loss: 2957.9517
AP SCORE：  0.7511439449966931
AUC SCORE:  0.7777423469387755
Epoch: 00, Loss: 2950.5913
Epoch: 01, Loss: 2950.5039
Epoch: 02, Loss: 2950.3521
Epoch: 03, Loss: 2950.1672
Epoch: 04, Loss: 2949.9761
AP SCORE：  0.7494372846599594
AUC SCORE:  0.7786989795918368
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 2949.7971
Epoch: 01, Loss: 2947.9812
Epoch: 02, Loss: 2944.5503
Epoch: 03, Loss: 2939.6990
Epoch: 04, Loss: 2933.6118
AP SCORE：  0.7522255519377739
AUC SCORE:  0.7796556122448979
Epoch: 00, Loss: 2926.4604
Epoch: 01, Loss: 2926.3777
Epoch: 02, Loss: 2926.2354
Epoch: 03, Loss: 2926.0630
Epoch: 04, Loss: 2925.8845
AP SCORE：  0.74954232246883
AUC SCORE:  0.7793367346938775
######################### 模块循环 ： 26 ##########################
Epoch: 00, Loss: 2925.7175
Epoch: 01, Loss: 2923.9507
Epoch: 02, Loss: 2920.6121
Epoch: 03, Loss: 2915.8911
Epoch: 04, 

Epoch: 04, Loss: 2601.9680
AP SCORE：  0.754499298185639
AUC SCORE:  0.78125
Epoch: 00, Loss: 2597.0684
Epoch: 01, Loss: 2597.0234
Epoch: 02, Loss: 2596.9521
Epoch: 03, Loss: 2596.8730
Epoch: 04, Loss: 2596.7981
AP SCORE：  0.7516951982158412
AUC SCORE:  0.7809311224489796
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 2596.7344
Epoch: 01, Loss: 2595.5166
Epoch: 02, Loss: 2593.2124
Epoch: 03, Loss: 2589.9507
Epoch: 04, Loss: 2585.8491
AP SCORE：  0.7542487154310126
AUC SCORE:  0.7815688775510204
Epoch: 00, Loss: 2581.0166
Epoch: 01, Loss: 2580.9714
Epoch: 02, Loss: 2580.8992
Epoch: 03, Loss: 2580.8181
Epoch: 04, Loss: 2580.7427
AP SCORE：  0.7529500141323113
AUC SCORE:  0.78125
######################### 模块循环 ： 44 ##########################
Epoch: 00, Loss: 2580.6790
Epoch: 01, Loss: 2579.4756
Epoch: 02, Loss: 2577.1997
Epoch: 03, Loss: 2573.9766
Epoch: 04, Loss: 2569.9243
AP SCORE：  0.7540009351070484
AUC SCORE:  0.7802933673469389
Epoch: 00, Loss: 2565.152

Epoch: 00, Loss: 2339.2275
Epoch: 01, Loss: 2339.2007
Epoch: 02, Loss: 2339.1577
Epoch: 03, Loss: 2339.1108
Epoch: 04, Loss: 2339.0686
AP SCORE：  0.745186969045114
AUC SCORE:  0.7815688775510204
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 2339.0359
Epoch: 01, Loss: 2338.0959
Epoch: 02, Loss: 2336.3179
Epoch: 03, Loss: 2333.7979
Epoch: 04, Loss: 2330.6279
AP SCORE：  0.7456956248218163
AUC SCORE:  0.7815688775510204
Epoch: 00, Loss: 2326.8918
Epoch: 01, Loss: 2326.8667
Epoch: 02, Loss: 2326.8257
Epoch: 03, Loss: 2326.7808
Epoch: 04, Loss: 2326.7405
AP SCORE：  0.7458625263247686
AUC SCORE:  0.7815688775510204
######################### 模块循环 ： 62 ##########################
Epoch: 00, Loss: 2326.7095
Epoch: 01, Loss: 2325.7876
Epoch: 02, Loss: 2324.0437
Epoch: 03, Loss: 2321.5728
Epoch: 04, Loss: 2318.4648
AP SCORE：  0.7470760818935532
AUC SCORE:  0.7822066326530612
Epoch: 00, Loss: 2314.8022
Epoch: 01, Loss: 2314.7778
Epoch: 02, Loss: 2314.7388
Epoch: 03,

Epoch: 01, Loss: 2138.7332
Epoch: 02, Loss: 2138.7000
Epoch: 03, Loss: 2138.6650
Epoch: 04, Loss: 2138.6355
AP SCORE：  0.7708154075560139
AUC SCORE:  0.7946428571428571
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 2138.6145
Epoch: 01, Loss: 2137.8591
Epoch: 02, Loss: 2136.4307
Epoch: 03, Loss: 2134.4067
Epoch: 04, Loss: 2131.8572
AP SCORE：  0.7736441305086095
AUC SCORE:  0.7943239795918366
Epoch: 00, Loss: 2128.8455
Epoch: 01, Loss: 2128.8242
Epoch: 02, Loss: 2128.7900
Epoch: 03, Loss: 2128.7542
Epoch: 04, Loss: 2128.7231
AP SCORE：  0.7739359582460894
AUC SCORE:  0.7943239795918366
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 2128.7007
Epoch: 01, Loss: 2127.9509
Epoch: 02, Loss: 2126.5322
Epoch: 03, Loss: 2124.5217
Epoch: 04, Loss: 2121.9883
AP SCORE：  0.7728081617666729
AUC SCORE:  0.7936862244897959
Epoch: 00, Loss: 2118.9944
Epoch: 01, Loss: 2118.9719
Epoch: 02, Loss: 2118.9368
Epoch: 03, Loss: 2118.8994
Epoch: 04

Epoch: 02, Loss: 1974.2787
Epoch: 03, Loss: 1974.2563
Epoch: 04, Loss: 1974.2395
AP SCORE：  0.7612188464369322
AUC SCORE:  0.7818877551020408
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 1974.2295
Epoch: 01, Loss: 1973.6041
Epoch: 02, Loss: 1972.4213
Epoch: 03, Loss: 1970.7437
Epoch: 04, Loss: 1968.6293
AP SCORE：  0.7616531179682621
AUC SCORE:  0.7825255102040816
Epoch: 00, Loss: 1966.1311
Epoch: 01, Loss: 1966.1166
Epoch: 02, Loss: 1966.0940
Epoch: 03, Loss: 1966.0712
Epoch: 04, Loss: 1966.0536
AP SCORE：  0.7610756668699777
AUC SCORE:  0.7818877551020408
######################### 模块循环 ： 98 ##########################
Epoch: 00, Loss: 1966.0432
Epoch: 01, Loss: 1965.4227
Epoch: 02, Loss: 1964.2489
Epoch: 03, Loss: 1962.5847
Epoch: 04, Loss: 1960.4873
AP SCORE：  0.7608188698257751
AUC SCORE:  0.78125
Epoch: 00, Loss: 1958.0090
Epoch: 01, Loss: 1957.9945
Epoch: 02, Loss: 1957.9717
Epoch: 03, Loss: 1957.9486
Epoch: 04, Loss: 1957.9309
AP SCORE：  0.7612437

In [9]:
context_embedding_dim = 12
importance_embedding_dim = 88
beta = 4.4   # 原来是4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
NEGATIVE_SAMPLE = 6
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)
    
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 5634.1895
Epoch: 01, Loss: 5572.3306
Epoch: 02, Loss: 5457.5049
Epoch: 03, Loss: 5307.5698
Epoch: 04, Loss: 5148.2407
AP SCORE：  0.5595346410580158
AUC SCORE:  0.5746173469387755
Epoch: 00, Loss: 4989.9189
Epoch: 01, Loss: 4988.6812
Epoch: 02, Loss: 4986.4902
Epoch: 03, Loss: 4983.7090
Epoch: 04, Loss: 4980.7056
AP SCORE：  0.5659619979813835
AUC SCORE:  0.6071428571428571
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4977.7710
Epoch: 01, Loss: 4939.3784
Epoch: 02, Loss: 4873.3208
Epoch: 03, Loss: 4790.6963
Epoch: 04, Loss: 4697.1787
AP SCORE：  0.5984368214808462
AUC SCORE:  0.6508290816326531
Epoch: 00, Loss: 4595.3633
Epoch: 01, Loss: 4592.0464
Epoch: 02, Loss: 4586.3887
Epoch: 03, Loss: 4580.0352
Epoch: 04, Loss: 4575.0352
AP SCORE：  0.6295040148736016
AUC SCORE:  0.6871811224489797
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4571.9038
Epo

Epoch: 00, Loss: 3186.9072
Epoch: 01, Loss: 3184.4734
Epoch: 02, Loss: 3179.8767
Epoch: 03, Loss: 3173.3867
Epoch: 04, Loss: 3165.2568
AP SCORE：  0.7310226539514559
AUC SCORE:  0.7369260204081631
Epoch: 00, Loss: 3155.7266
Epoch: 01, Loss: 3155.6196
Epoch: 02, Loss: 3155.4307
Epoch: 03, Loss: 3155.1895
Epoch: 04, Loss: 3154.9248
AP SCORE：  0.7331031793589745
AUC SCORE:  0.7401147959183674
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 3154.6626
Epoch: 01, Loss: 3152.3359
Epoch: 02, Loss: 3147.9404
Epoch: 03, Loss: 3141.7275
Epoch: 04, Loss: 3133.9407
AP SCORE：  0.7345495188513546
AUC SCORE:  0.7420280612244898
Epoch: 00, Loss: 3124.8140
Epoch: 01, Loss: 3124.7158
Epoch: 02, Loss: 3124.5459
Epoch: 03, Loss: 3124.3357
Epoch: 04, Loss: 3124.1179
AP SCORE：  0.7338346358658348
AUC SCORE:  0.741390306122449
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 3123.9150
Epoch: 01, Loss: 3121.7061
Epoch: 02, Loss: 3117.5359
Epoch: 03,

Epoch: 01, Loss: 2759.0112
Epoch: 02, Loss: 2756.4717
Epoch: 03, Loss: 2752.8711
Epoch: 04, Loss: 2748.3379
AP SCORE：  0.7511910329220474
AUC SCORE:  0.7509566326530612
Epoch: 00, Loss: 2742.9888
Epoch: 01, Loss: 2742.9478
Epoch: 02, Loss: 2742.8772
Epoch: 03, Loss: 2742.7927
Epoch: 04, Loss: 2742.7083
AP SCORE：  0.7510750993092953
AUC SCORE:  0.7503188775510203
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2742.6333
Epoch: 01, Loss: 2741.3140
Epoch: 02, Loss: 2738.8167
Epoch: 03, Loss: 2735.2749
Epoch: 04, Loss: 2730.8142
AP SCORE：  0.7516766439002879
AUC SCORE:  0.7509566326530612
Epoch: 00, Loss: 2725.5488
Epoch: 01, Loss: 2725.5090
Epoch: 02, Loss: 2725.4409
Epoch: 03, Loss: 2725.3606
Epoch: 04, Loss: 2725.2820
AP SCORE：  0.752162387437214
AUC SCORE:  0.7512755102040816
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2725.2144
Epoch: 01, Loss: 2723.9143
Epoch: 02, Loss: 2721.4536
Epoch: 03, Loss: 2717.9646
Epoch: 04,

Epoch: 02, Loss: 2481.3838
Epoch: 03, Loss: 2478.6272
Epoch: 04, Loss: 2475.1526
AP SCORE：  0.7498383020806662
AUC SCORE:  0.7586096938775511
Epoch: 00, Loss: 2471.0498
Epoch: 01, Loss: 2471.0256
Epoch: 02, Loss: 2470.9856
Epoch: 03, Loss: 2470.9390
Epoch: 04, Loss: 2470.8945
AP SCORE：  0.7498383020806662
AUC SCORE:  0.7586096938775511
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 2470.8560
Epoch: 01, Loss: 2469.8455
Epoch: 02, Loss: 2467.9321
Epoch: 03, Loss: 2465.2175
Epoch: 04, Loss: 2461.7957
AP SCORE：  0.7473747112383028
AUC SCORE:  0.7595663265306123
Epoch: 00, Loss: 2457.7522
Epoch: 01, Loss: 2457.7275
Epoch: 02, Loss: 2457.6863
Epoch: 03, Loss: 2457.6384
Epoch: 04, Loss: 2457.5920
AP SCORE：  0.7477168613154656
AUC SCORE:  0.7598852040816326
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 2457.5515
Epoch: 01, Loss: 2456.5544
Epoch: 02, Loss: 2454.6670
Epoch: 03, Loss: 2451.9893
Epoch: 04, Loss: 2448.6130
AP SCORE：

Epoch: 03, Loss: 2262.5940
Epoch: 04, Loss: 2259.8760
AP SCORE：  0.7561484768840162
AUC SCORE:  0.775829081632653
Epoch: 00, Loss: 2256.6653
Epoch: 01, Loss: 2256.6394
Epoch: 02, Loss: 2256.5974
Epoch: 03, Loss: 2256.5486
Epoch: 04, Loss: 2256.5020
AP SCORE：  0.7562897517483923
AUC SCORE:  0.7761479591836735
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 2256.4602
Epoch: 01, Loss: 2255.6636
Epoch: 02, Loss: 2254.1562
Epoch: 03, Loss: 2252.0181
Epoch: 04, Loss: 2249.3230
AP SCORE：  0.7563885478149303
AUC SCORE:  0.7761479591836734
Epoch: 00, Loss: 2246.1389
Epoch: 01, Loss: 2246.1133
Epoch: 02, Loss: 2246.0713
Epoch: 03, Loss: 2246.0220
Epoch: 04, Loss: 2245.9749
AP SCORE：  0.7564981969377372
AUC SCORE:  0.7764668367346939
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 2245.9331
Epoch: 01, Loss: 2245.1411
Epoch: 02, Loss: 2243.6421
Epoch: 03, Loss: 2241.5146
Epoch: 04, Loss: 2238.8330
AP SCORE：  0.756279295213668
AUC SCOR

...

Epoch: 00, Loss: 3018.6335
Epoch: 01, Loss: 3018.5591
Epoch: 02, Loss: 3018.4292
Epoch: 03, Loss: 3018.2681
Epoch: 04, Loss: 3018.0952
AP SCORE：  0.7261897422402387
AUC SCORE:  0.6957908163265306
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 3017.9277
Epoch: 01, Loss: 3015.9980
Epoch: 02, Loss: 3012.3494
Epoch: 03, Loss: 3007.1833
Epoch: 04, Loss: 3000.6904
AP SCORE：  0.7245333959465319
AUC SCORE:  0.6941964285714286
Epoch: 00, Loss: 2993.0444
Epoch: 01, Loss: 2992.9768
Epoch: 02, Loss: 2992.8584
Epoch: 03, Loss: 2992.7107
Epoch: 04, Loss: 2992.5522
AP SCORE：  0.7246437732509654
AUC SCORE:  0.6941964285714286
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 2992.3975
Epoch: 01, Loss: 2990.5237
Epoch: 02, Loss: 2986.9773
Epoch: 03, Loss: 2981.9541
Epoch: 04, Loss: 2975.6350
AP SCORE：  0.7216591960532903
AUC SCORE:  0.6919642857142857
Epoch: 00, Loss: 2968.1899
Epoch: 01, Loss: 2968.1218
Epoch: 02, Loss: 2968.0034
Epoch: 03

Epoch: 01, Loss: 2644.5383
Epoch: 02, Loss: 2644.4802
Epoch: 03, Loss: 2644.4119
Epoch: 04, Loss: 2644.3450
AP SCORE：  0.7150151535163136
AUC SCORE:  0.6973852040816326
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 2644.2866
Epoch: 01, Loss: 2643.0415
Epoch: 02, Loss: 2640.6865
Epoch: 03, Loss: 2637.3501
Epoch: 04, Loss: 2633.1521
AP SCORE：  0.7151720390966819
AUC SCORE:  0.6980229591836734
Epoch: 00, Loss: 2628.2026
Epoch: 01, Loss: 2628.1697
Epoch: 02, Loss: 2628.1140
Epoch: 03, Loss: 2628.0479
Epoch: 04, Loss: 2627.9817
AP SCORE：  0.7148808500665752
AUC SCORE:  0.6973852040816327
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 2627.9221
Epoch: 01, Loss: 2626.7061
Epoch: 02, Loss: 2624.4041
Epoch: 03, Loss: 2621.1406
Epoch: 04, Loss: 2617.0312
AP SCORE：  0.7140963557453259
AUC SCORE:  0.6967474489795918
Epoch: 00, Loss: 2612.1836
Epoch: 01, Loss: 2612.1487
Epoch: 02, Loss: 2612.0898
Epoch: 03, Loss: 2612.0195
Epoch: 04

Epoch: 02, Loss: 2388.1458
Epoch: 03, Loss: 2388.0996
Epoch: 04, Loss: 2388.0552
AP SCORE：  0.7309714068736343
AUC SCORE:  0.7114158163265306
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 2388.0164
Epoch: 01, Loss: 2387.0815
Epoch: 02, Loss: 2385.3113
Epoch: 03, Loss: 2382.8005
Epoch: 04, Loss: 2379.6350
AP SCORE：  0.7316395577313554
AUC SCORE:  0.711734693877551
Epoch: 00, Loss: 2375.8938
Epoch: 01, Loss: 2375.8701
Epoch: 02, Loss: 2375.8296
Epoch: 03, Loss: 2375.7834
Epoch: 04, Loss: 2375.7397
AP SCORE：  0.7328534921234006
AUC SCORE:  0.7133290816326531
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 2375.7019
Epoch: 01, Loss: 2374.7783
Epoch: 02, Loss: 2373.0298
Epoch: 03, Loss: 2370.5496
Epoch: 04, Loss: 2367.4221
AP SCORE：  0.7321282233397918
AUC SCORE:  0.7126913265306123
Epoch: 00, Loss: 2363.7258
Epoch: 01, Loss: 2363.7024
Epoch: 02, Loss: 2363.6643
Epoch: 03, Loss: 2363.6213
Epoch: 04, Loss: 2363.5811
AP SCORE： 

Epoch: 03, Loss: 2185.6418
Epoch: 04, Loss: 2185.6047
AP SCORE：  0.7424428536994045
AUC SCORE:  0.7267219387755102
######################### 模块循环 ： 78 ##########################
Epoch: 00, Loss: 2185.5757
Epoch: 01, Loss: 2184.7910
Epoch: 02, Loss: 2183.3062
Epoch: 03, Loss: 2181.2000
Epoch: 04, Loss: 2178.5457
AP SCORE：  0.7419587838088221
AUC SCORE:  0.7270408163265306
Epoch: 00, Loss: 2175.4094
Epoch: 01, Loss: 2175.3909
Epoch: 02, Loss: 2175.3608
Epoch: 03, Loss: 2175.3286
Epoch: 04, Loss: 2175.3015
AP SCORE：  0.7419134117666094
AUC SCORE:  0.7276785714285714
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 2175.2822
Epoch: 01, Loss: 2174.5066
Epoch: 02, Loss: 2173.0398
Epoch: 03, Loss: 2170.9590
Epoch: 04, Loss: 2168.3364
AP SCORE：  0.74150169024194
AUC SCORE:  0.7286352040816326
Epoch: 00, Loss: 2165.2380
Epoch: 01, Loss: 2165.2207
Epoch: 02, Loss: 2165.1931
Epoch: 03, Loss: 2165.1641
Epoch: 04, Loss: 2165.1394
AP SCORE：  0.7413471918228541
AUC SCOR

Epoch: 04, Loss: 2018.6809
AP SCORE：  0.736431123549784
AUC SCORE:  0.7305484693877551
######################### 模块循环 ： 96 ##########################
Epoch: 00, Loss: 2018.6660
Epoch: 01, Loss: 2018.0281
Epoch: 02, Loss: 2016.8207
Epoch: 03, Loss: 2015.1094
Epoch: 04, Loss: 2012.9531
AP SCORE：  0.734745203717105
AUC SCORE:  0.7295918367346939
Epoch: 00, Loss: 2010.4077
Epoch: 01, Loss: 2010.3937
Epoch: 02, Loss: 2010.3716
Epoch: 03, Loss: 2010.3491
Epoch: 04, Loss: 2010.3309
AP SCORE：  0.7348732669906072
AUC SCORE:  0.7299107142857143
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 2010.3186
Epoch: 01, Loss: 2009.6906
Epoch: 02, Loss: 2008.5023
Epoch: 03, Loss: 2006.8181
Epoch: 04, Loss: 2004.6973
AP SCORE：  0.7345760307847743
AUC SCORE:  0.7295918367346939
Epoch: 00, Loss: 2002.1927
Epoch: 01, Loss: 2002.1794
Epoch: 02, Loss: 2002.1586
Epoch: 03, Loss: 2002.1375
Epoch: 04, Loss: 2002.1205
AP SCORE：  0.7347491851003912
AUC SCORE:  0.7299107142857143
####

In [10]:
context_embedding_dim = 12
importance_embedding_dim = 88
beta = 4.2   # 原来是4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
NEGATIVE_SAMPLE = 6
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)
    
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 5412.9834
Epoch: 01, Loss: 5361.0229
Epoch: 02, Loss: 5265.0127
Epoch: 03, Loss: 5140.6841
Epoch: 04, Loss: 5006.6973
AP SCORE：  0.5421866565768954
AUC SCORE:  0.47225765306122447
Epoch: 00, Loss: 4870.7764
Epoch: 01, Loss: 4869.5723
Epoch: 02, Loss: 4867.3774
Epoch: 03, Loss: 4864.4624
Epoch: 04, Loss: 4861.1562
AP SCORE：  0.5586703739484662
AUC SCORE:  0.48756377551020413
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4857.9302
Epoch: 01, Loss: 4822.8486
Epoch: 02, Loss: 4763.3926
Epoch: 03, Loss: 4689.5396
Epoch: 04, Loss: 4604.1270
AP SCORE：  0.5899008847126462
AUC SCORE:  0.5280612244897959
Epoch: 00, Loss: 4510.2075
Epoch: 01, Loss: 4505.8892
Epoch: 02, Loss: 4499.5664
Epoch: 03, Loss: 4493.3369
Epoch: 04, Loss: 4487.9209
AP SCORE：  0.5726588193275508
AUC SCORE:  0.5500637755102041
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4484.0400
E

Epoch: 00, Loss: 3147.2197
Epoch: 01, Loss: 3144.8547
Epoch: 02, Loss: 3140.3892
Epoch: 03, Loss: 3134.0786
Epoch: 04, Loss: 3126.1687
AP SCORE：  0.6265469130751662
AUC SCORE:  0.6690051020408163
Epoch: 00, Loss: 3116.8894
Epoch: 01, Loss: 3116.7976
Epoch: 02, Loss: 3116.6328
Epoch: 03, Loss: 3116.4165
Epoch: 04, Loss: 3116.1729
AP SCORE：  0.6277987650797682
AUC SCORE:  0.6705994897959184
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 3115.9189
Epoch: 01, Loss: 3113.6746
Epoch: 02, Loss: 3109.4331
Epoch: 03, Loss: 3103.4363
Epoch: 04, Loss: 3095.9124
AP SCORE：  0.6296413921314384
AUC SCORE:  0.671875
Epoch: 00, Loss: 3087.0750
Epoch: 01, Loss: 3086.9863
Epoch: 02, Loss: 3086.8252
Epoch: 03, Loss: 3086.6152
Epoch: 04, Loss: 3086.3770
AP SCORE：  0.631909548029875
AUC SCORE:  0.6731505102040816
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 3086.1289
Epoch: 01, Loss: 3083.9854
Epoch: 02, Loss: 3079.9338
Epoch: 03, Loss: 307

Epoch: 02, Loss: 2722.1506
Epoch: 03, Loss: 2718.5439
Epoch: 04, Loss: 2714.0024
AP SCORE：  0.6841471055902097
AUC SCORE:  0.7075892857142857
Epoch: 00, Loss: 2708.6438
Epoch: 01, Loss: 2708.5935
Epoch: 02, Loss: 2708.5059
Epoch: 03, Loss: 2708.3967
Epoch: 04, Loss: 2708.2803
AP SCORE：  0.683650957204193
AUC SCORE:  0.7069515306122449
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2708.1682
Epoch: 01, Loss: 2706.8496
Epoch: 02, Loss: 2704.3540
Epoch: 03, Loss: 2700.8157
Epoch: 04, Loss: 2696.3601
AP SCORE：  0.6837968352291746
AUC SCORE:  0.7066326530612245
Epoch: 00, Loss: 2691.1016
Epoch: 01, Loss: 2691.0542
Epoch: 02, Loss: 2690.9729
Epoch: 03, Loss: 2690.8726
Epoch: 04, Loss: 2690.7676
AP SCORE：  0.6835896781718657
AUC SCORE:  0.7063137755102041
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2690.6689
Epoch: 01, Loss: 2689.3730
Epoch: 02, Loss: 2686.9199
Epoch: 03, Loss: 2683.4426
Epoch: 04, Loss: 2679.0627
AP SCORE： 

Epoch: 03, Loss: 2440.0574
Epoch: 04, Loss: 2436.5527
AP SCORE：  0.694423975354155
AUC SCORE:  0.7044005102040817
Epoch: 00, Loss: 2432.4165
Epoch: 01, Loss: 2432.3833
Epoch: 02, Loss: 2432.3293
Epoch: 03, Loss: 2432.2656
Epoch: 04, Loss: 2432.2043
AP SCORE：  0.6943022546530438
AUC SCORE:  0.7040816326530612
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 2432.1531
Epoch: 01, Loss: 2431.1309
Epoch: 02, Loss: 2429.1960
Epoch: 03, Loss: 2426.4543
Epoch: 04, Loss: 2423.0032
AP SCORE：  0.694423975354155
AUC SCORE:  0.7044005102040817
Epoch: 00, Loss: 2418.9319
Epoch: 01, Loss: 2418.9006
Epoch: 02, Loss: 2418.8494
Epoch: 03, Loss: 2418.7891
Epoch: 04, Loss: 2418.7302
AP SCORE：  0.6929521296227946
AUC SCORE:  0.7040816326530612
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 2418.6797
Epoch: 01, Loss: 2417.6704
Epoch: 02, Loss: 2415.7593
Epoch: 03, Loss: 2413.0476
Epoch: 04, Loss: 2409.6282
AP SCORE：  0.6915471586460269
AUC SCOR

Epoch: 04, Loss: 2223.8076
AP SCORE：  0.6711818318068412
AUC SCORE:  0.6951530612244898
Epoch: 00, Loss: 2220.6658
Epoch: 01, Loss: 2220.6436
Epoch: 02, Loss: 2220.6069
Epoch: 03, Loss: 2220.5635
Epoch: 04, Loss: 2220.5203
AP SCORE：  0.6712812222123541
AUC SCORE:  0.6954719387755103
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 2220.4844
Epoch: 01, Loss: 2219.7063
Epoch: 02, Loss: 2218.2339
Epoch: 03, Loss: 2216.1436
Epoch: 04, Loss: 2213.5078
AP SCORE：  0.6717427168830574
AUC SCORE:  0.6957908163265307
Epoch: 00, Loss: 2210.3918
Epoch: 01, Loss: 2210.3704
Epoch: 02, Loss: 2210.3352
Epoch: 03, Loss: 2210.2954
Epoch: 04, Loss: 2210.2573
AP SCORE：  0.672501353671106
AUC SCORE:  0.6961096938775511
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 2210.2268
Epoch: 01, Loss: 2209.4568
Epoch: 02, Loss: 2207.9978
Epoch: 03, Loss: 2205.9282
Epoch: 04, Loss: 2203.3191
AP SCORE：  0.6725595168375161
AUC SCORE:  0.6961096938775511
Epo

...

Epoch: 02, Loss: 3006.9321
Epoch: 03, Loss: 3006.6973
Epoch: 04, Loss: 3006.4346
AP SCORE：  0.6887257202216295
AUC SCORE:  0.7248086734693878
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 3006.1692
Epoch: 01, Loss: 3004.2891
Epoch: 02, Loss: 3000.7373
Epoch: 03, Loss: 2995.7151
Epoch: 04, Loss: 2989.4153
AP SCORE：  0.6890422737386046
AUC SCORE:  0.7248086734693877
Epoch: 00, Loss: 2982.0166
Epoch: 01, Loss: 2981.9287
Epoch: 02, Loss: 2981.7778
Epoch: 03, Loss: 2981.5940
Epoch: 04, Loss: 2981.4092
AP SCORE：  0.6898746502076616
AUC SCORE:  0.7254464285714286
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 2981.2441
Epoch: 01, Loss: 2979.4307
Epoch: 02, Loss: 2976.0046
Epoch: 03, Loss: 2971.1616
Epoch: 04, Loss: 2965.0862
AP SCORE：  0.6936769867160568
AUC SCORE:  0.727359693877551
Epoch: 00, Loss: 2957.9487
Epoch: 01, Loss: 2957.8762
Epoch: 02, Loss: 2957.7517
Epoch: 03, Loss: 2957.6008
Epoch: 04, Loss: 2957.4485
AP SCORE： 

Epoch: 03, Loss: 2647.4656
Epoch: 04, Loss: 2647.3911
AP SCORE：  0.7290489521886003
AUC SCORE:  0.7442602040816326
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 2647.3254
Epoch: 01, Loss: 2646.0940
Epoch: 02, Loss: 2643.7632
Epoch: 03, Loss: 2640.4587
Epoch: 04, Loss: 2636.2988
AP SCORE：  0.7308743160626978
AUC SCORE:  0.7452168367346939
Epoch: 00, Loss: 2631.3904
Epoch: 01, Loss: 2631.3577
Epoch: 02, Loss: 2631.3030
Epoch: 03, Loss: 2631.2395
Epoch: 04, Loss: 2631.1775
AP SCORE：  0.7309650542985154
AUC SCORE:  0.7448979591836734
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 2631.1221
Epoch: 01, Loss: 2629.9160
Epoch: 02, Loss: 2627.6318
Epoch: 03, Loss: 2624.3933
Epoch: 04, Loss: 2620.3127
AP SCORE：  0.7333793280044236
AUC SCORE:  0.7442602040816326
Epoch: 00, Loss: 2615.4966
Epoch: 01, Loss: 2615.4661
Epoch: 02, Loss: 2615.4146
Epoch: 03, Loss: 2615.3555
Epoch: 04, Loss: 2615.2974
AP SCORE：  0.7331176780819368
AUC SC

Epoch: 04, Loss: 2390.8687
AP SCORE：  0.7331940293402369
AUC SCORE:  0.7321428571428571
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 2390.8228
Epoch: 01, Loss: 2389.8931
Epoch: 02, Loss: 2388.1321
Epoch: 03, Loss: 2385.6350
Epoch: 04, Loss: 2382.4900
AP SCORE：  0.733615786195434
AUC SCORE:  0.7330994897959183
Epoch: 00, Loss: 2378.7800
Epoch: 01, Loss: 2378.7524
Epoch: 02, Loss: 2378.7114
Epoch: 03, Loss: 2378.6663
Epoch: 04, Loss: 2378.6243
AP SCORE：  0.7331785536378573
AUC SCORE:  0.7323022959183674
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 2378.5896
Epoch: 01, Loss: 2377.6731
Epoch: 02, Loss: 2375.9392
Epoch: 03, Loss: 2373.4817
Epoch: 04, Loss: 2370.3901
AP SCORE：  0.7352817805909858
AUC SCORE:  0.7327806122448979
Epoch: 00, Loss: 2366.7471
Epoch: 01, Loss: 2366.7222
Epoch: 02, Loss: 2366.6843
Epoch: 03, Loss: 2366.6431
Epoch: 04, Loss: 2366.6038
AP SCORE：  0.7375196504474176
AUC SCORE:  0.7324617346938775
###

Epoch: 00, Loss: 2195.6414
Epoch: 01, Loss: 2194.9067
Epoch: 02, Loss: 2193.5151
Epoch: 03, Loss: 2191.5425
Epoch: 04, Loss: 2189.0562
AP SCORE：  0.7360304468293498
AUC SCORE:  0.7350127551020408
Epoch: 00, Loss: 2186.1189
Epoch: 01, Loss: 2186.0977
Epoch: 02, Loss: 2186.0664
Epoch: 03, Loss: 2186.0317
Epoch: 04, Loss: 2185.9990
AP SCORE：  0.738876340431546
AUC SCORE:  0.735969387755102
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 2185.9734
Epoch: 01, Loss: 2185.2456
Epoch: 02, Loss: 2183.8687
Epoch: 03, Loss: 2181.9150
Epoch: 04, Loss: 2179.4546
AP SCORE：  0.7386033624916538
AUC SCORE:  0.7353316326530611
Epoch: 00, Loss: 2176.5496
Epoch: 01, Loss: 2176.5293
Epoch: 02, Loss: 2176.4980
Epoch: 03, Loss: 2176.4641
Epoch: 04, Loss: 2176.4321
AP SCORE：  0.7384878034475234
AUC SCORE:  0.7350127551020409
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 2176.4072
Epoch: 01, Loss: 2175.6877
Epoch: 02, Loss: 2174.3267
Epoch: 03, 

Epoch: 01, Loss: 2035.1675
Epoch: 02, Loss: 2034.0044
Epoch: 03, Loss: 2032.3553
Epoch: 04, Loss: 2030.2786
AP SCORE：  0.7279551183974925
AUC SCORE:  0.728954081632653
Epoch: 00, Loss: 2027.8268
Epoch: 01, Loss: 2027.8118
Epoch: 02, Loss: 2027.7903
Epoch: 03, Loss: 2027.7681
Epoch: 04, Loss: 2027.7498
AP SCORE：  0.727405163965959
AUC SCORE:  0.7283163265306122
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 2027.7380
Epoch: 01, Loss: 2027.1313
Epoch: 02, Loss: 2025.9823
Epoch: 03, Loss: 2024.3540
Epoch: 04, Loss: 2022.3025
AP SCORE：  0.7288354641495384
AUC SCORE:  0.7292729591836735
Epoch: 00, Loss: 2019.8800
Epoch: 01, Loss: 2019.8649
Epoch: 02, Loss: 2019.8430
Epoch: 03, Loss: 2019.8209
Epoch: 04, Loss: 2019.8027
AP SCORE：  0.7278897828467142
AUC SCORE:  0.7286352040816326
######################### 模块循环 ： 98 ##########################
Epoch: 00, Loss: 2019.7914
Epoch: 01, Loss: 2019.1906
Epoch: 02, Loss: 2018.0540
Epoch: 03, Loss: 2016.4429
Epoch: 04, 

In [11]:
context_embedding_dim = 12
importance_embedding_dim = 88
beta = 4.4   # 原来是4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
NEGATIVE_SAMPLE = 6
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)
    
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 5611.3076
Epoch: 01, Loss: 5548.5752
Epoch: 02, Loss: 5438.2559
Epoch: 03, Loss: 5299.2979
Epoch: 04, Loss: 5144.6465
AP SCORE：  0.5982312520055652
AUC SCORE:  0.5293367346938775
Epoch: 00, Loss: 4982.5576
Epoch: 01, Loss: 4981.5835
Epoch: 02, Loss: 4979.7842
Epoch: 03, Loss: 4977.3325
Epoch: 04, Loss: 4974.4067
AP SCORE：  0.6095491895995301
AUC SCORE:  0.5436862244897959
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4971.2026
Epoch: 01, Loss: 4934.8799
Epoch: 02, Loss: 4873.4233
Epoch: 03, Loss: 4794.5332
Epoch: 04, Loss: 4702.6543
AP SCORE：  0.6375343399923206
AUC SCORE:  0.5672831632653061
Epoch: 00, Loss: 4602.4243
Epoch: 01, Loss: 4599.1655
Epoch: 02, Loss: 4593.4800
Epoch: 03, Loss: 4587.3105
Epoch: 04, Loss: 4582.8516
AP SCORE：  0.6203267739027057
AUC SCORE:  0.5816326530612245
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4579.6074
Epo

Epoch: 00, Loss: 3199.3284
Epoch: 01, Loss: 3196.8918
Epoch: 02, Loss: 3192.2893
Epoch: 03, Loss: 3185.7856
Epoch: 04, Loss: 3177.6333
AP SCORE：  0.660300845243189
AUC SCORE:  0.6865433673469388
Epoch: 00, Loss: 3168.0698
Epoch: 01, Loss: 3167.9575
Epoch: 02, Loss: 3167.7588
Epoch: 03, Loss: 3167.5051
Epoch: 04, Loss: 3167.2288
AP SCORE：  0.6602072207831774
AUC SCORE:  0.6868622448979591
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 3166.9585
Epoch: 01, Loss: 3164.6226
Epoch: 02, Loss: 3160.2139
Epoch: 03, Loss: 3153.9907
Epoch: 04, Loss: 3146.2058
AP SCORE：  0.6607350006292839
AUC SCORE:  0.6871811224489796
Epoch: 00, Loss: 3137.0984
Epoch: 01, Loss: 3137.0049
Epoch: 02, Loss: 3136.8430
Epoch: 03, Loss: 3136.6436
Epoch: 04, Loss: 3136.4395
AP SCORE：  0.6618989901485366
AUC SCORE:  0.6875
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 3136.2544
Epoch: 01, Loss: 3134.0562
Epoch: 02, Loss: 3129.9087
Epoch: 03, Loss: 3124.

Epoch: 02, Loss: 2772.1992
Epoch: 03, Loss: 2768.4661
Epoch: 04, Loss: 2763.7637
AP SCORE：  0.6936312513118529
AUC SCORE:  0.6980229591836734
Epoch: 00, Loss: 2758.2151
Epoch: 01, Loss: 2758.1497
Epoch: 02, Loss: 2758.0388
Epoch: 03, Loss: 2757.9060
Epoch: 04, Loss: 2757.7739
AP SCORE：  0.6944461571554548
AUC SCORE:  0.6992984693877552
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2757.6587
Epoch: 01, Loss: 2756.2751
Epoch: 02, Loss: 2753.6589
Epoch: 03, Loss: 2749.9573
Epoch: 04, Loss: 2745.3066
AP SCORE：  0.694947253372119
AUC SCORE:  0.7002551020408164
Epoch: 00, Loss: 2739.8357
Epoch: 01, Loss: 2739.7808
Epoch: 02, Loss: 2739.6868
Epoch: 03, Loss: 2739.5718
Epoch: 04, Loss: 2739.4548
AP SCORE：  0.6948258544138776
AUC SCORE:  0.7005739795918368
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2739.3474
Epoch: 01, Loss: 2737.9976
Epoch: 02, Loss: 2735.4438
Epoch: 03, Loss: 2731.8284
Epoch: 04, Loss: 2727.2844
AP SCORE： 

Epoch: 03, Loss: 2487.2214
Epoch: 04, Loss: 2483.7947
AP SCORE：  0.6950967929094991
AUC SCORE:  0.6980229591836735
Epoch: 00, Loss: 2479.7488
Epoch: 01, Loss: 2479.7261
Epoch: 02, Loss: 2479.6880
Epoch: 03, Loss: 2479.6460
Epoch: 04, Loss: 2479.6086
AP SCORE：  0.6949844770644247
AUC SCORE:  0.6980229591836735
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 2479.5806
Epoch: 01, Loss: 2478.5828
Epoch: 02, Loss: 2476.6946
Epoch: 03, Loss: 2474.0171
Epoch: 04, Loss: 2470.6440
AP SCORE：  0.695768092812711
AUC SCORE:  0.6989795918367347
Epoch: 00, Loss: 2466.6624
Epoch: 01, Loss: 2466.6392
Epoch: 02, Loss: 2466.6011
Epoch: 03, Loss: 2466.5596
Epoch: 04, Loss: 2466.5225
AP SCORE：  0.6953027030499226
AUC SCORE:  0.6980229591836735
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 2466.4941
Epoch: 01, Loss: 2465.5112
Epoch: 02, Loss: 2463.6511
Epoch: 03, Loss: 2461.0129
Epoch: 04, Loss: 2457.6887
AP SCORE：  0.6948426519975912
AUC SCO

Epoch: 04, Loss: 2271.8459
AP SCORE：  0.6991051763262991
AUC SCORE:  0.7018494897959183
Epoch: 00, Loss: 2268.7163
Epoch: 01, Loss: 2268.6951
Epoch: 02, Loss: 2268.6611
Epoch: 03, Loss: 2268.6230
Epoch: 04, Loss: 2268.5879
AP SCORE：  0.6987659353667574
AUC SCORE:  0.7012117346938775
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 2268.5586
Epoch: 01, Loss: 2267.7842
Epoch: 02, Loss: 2266.3186
Epoch: 03, Loss: 2264.2415
Epoch: 04, Loss: 2261.6235
AP SCORE：  0.6986890406321399
AUC SCORE:  0.700892857142857
Epoch: 00, Loss: 2258.5310
Epoch: 01, Loss: 2258.5088
Epoch: 02, Loss: 2258.4722
Epoch: 03, Loss: 2258.4304
Epoch: 04, Loss: 2258.3909
AP SCORE：  0.6982311651742644
AUC SCORE:  0.7005739795918368
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 2258.3562
Epoch: 01, Loss: 2257.5886
Epoch: 02, Loss: 2256.1362
Epoch: 03, Loss: 2254.0759
Epoch: 04, Loss: 2251.4785
AP SCORE：  0.6995384844477252
AUC SCORE:  0.7002551020408163
Epo

...

Epoch: 02, Loss: 3024.6736
Epoch: 03, Loss: 3019.5933
Epoch: 04, Loss: 3013.2507
AP SCORE：  0.6115557138647593
AUC SCORE:  0.6042729591836735
Epoch: 00, Loss: 3005.8403
Epoch: 01, Loss: 3005.7507
Epoch: 02, Loss: 3005.6025
Epoch: 03, Loss: 3005.4309
Epoch: 04, Loss: 3005.2627
AP SCORE：  0.6112994616623778
AUC SCORE:  0.6049107142857143
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 3005.1072
Epoch: 01, Loss: 3003.2825
Epoch: 02, Loss: 2999.8403
Epoch: 03, Loss: 2994.9861
Epoch: 04, Loss: 2988.9124
AP SCORE：  0.611256064577998
AUC SCORE:  0.6055484693877551
Epoch: 00, Loss: 2981.7983
Epoch: 01, Loss: 2981.7158
Epoch: 02, Loss: 2981.5771
Epoch: 03, Loss: 2981.4126
Epoch: 04, Loss: 2981.2466
AP SCORE：  0.6113265362913886
AUC SCORE:  0.6061862244897959
######################### 模块循环 ： 26 ##########################
Epoch: 00, Loss: 2981.0913
Epoch: 01, Loss: 2979.3406
Epoch: 02, Loss: 2976.0361
Epoch: 03, Loss: 2971.3699
Epoch: 04, Loss: 2965.5227
AP SCORE： 

Epoch: 04, Loss: 2664.9731
AP SCORE：  0.6164333564414343
AUC SCORE:  0.6192602040816326
Epoch: 00, Loss: 2660.2229
Epoch: 01, Loss: 2660.1838
Epoch: 02, Loss: 2660.1201
Epoch: 03, Loss: 2660.0461
Epoch: 04, Loss: 2659.9739
AP SCORE：  0.6172441542454044
AUC SCORE:  0.6195790816326531
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 2659.9111
Epoch: 01, Loss: 2658.7341
Epoch: 02, Loss: 2656.5071
Epoch: 03, Loss: 2653.3506
Epoch: 04, Loss: 2649.3779
AP SCORE：  0.6141433124572022
AUC SCORE:  0.620216836734694
Epoch: 00, Loss: 2644.6919
Epoch: 01, Loss: 2644.6531
Epoch: 02, Loss: 2644.5896
Epoch: 03, Loss: 2644.5161
Epoch: 04, Loss: 2644.4441
AP SCORE：  0.6147399755901859
AUC SCORE:  0.6205357142857143
######################### 模块循环 ： 44 ##########################
Epoch: 00, Loss: 2644.3804
Epoch: 01, Loss: 2643.2197
Epoch: 02, Loss: 2641.0234
Epoch: 03, Loss: 2637.9099
Epoch: 04, Loss: 2633.9897
AP SCORE：  0.6149012985250817
AUC SCORE:  0.6195790816326531
Epo

Epoch: 00, Loss: 2410.0220
Epoch: 01, Loss: 2409.9917
Epoch: 02, Loss: 2409.9436
Epoch: 03, Loss: 2409.8904
Epoch: 04, Loss: 2409.8411
AP SCORE：  0.6063767323050336
AUC SCORE:  0.6214923469387754
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 2409.8010
Epoch: 01, Loss: 2408.8594
Epoch: 02, Loss: 2407.0779
Epoch: 03, Loss: 2404.5544
Epoch: 04, Loss: 2401.3787
AP SCORE：  0.6053201700655652
AUC SCORE:  0.6205357142857142
Epoch: 00, Loss: 2397.6335
Epoch: 01, Loss: 2397.6062
Epoch: 02, Loss: 2397.5645
Epoch: 03, Loss: 2397.5186
Epoch: 04, Loss: 2397.4766
AP SCORE：  0.608835415449433
AUC SCORE:  0.6208545918367347
######################### 模块循环 ： 62 ##########################
Epoch: 00, Loss: 2397.4436
Epoch: 01, Loss: 2396.5181
Epoch: 02, Loss: 2394.7681
Epoch: 03, Loss: 2392.2881
Epoch: 04, Loss: 2389.1675
AP SCORE：  0.608396996200406
AUC SCORE:  0.6208545918367347
Epoch: 00, Loss: 2385.4861
Epoch: 01, Loss: 2385.4602
Epoch: 02, Loss: 2385.4202
Epoch: 03, 

Epoch: 01, Loss: 2211.1094
Epoch: 02, Loss: 2211.0835
Epoch: 03, Loss: 2211.0574
Epoch: 04, Loss: 2211.0366
AP SCORE：  0.5994697069314321
AUC SCORE:  0.6151147959183674
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 2211.0229
Epoch: 01, Loss: 2210.2671
Epoch: 02, Loss: 2208.8384
Epoch: 03, Loss: 2206.8118
Epoch: 04, Loss: 2204.2578
AP SCORE：  0.5990612112919507
AUC SCORE:  0.6138392857142858
Epoch: 00, Loss: 2201.2412
Epoch: 01, Loss: 2201.2241
Epoch: 02, Loss: 2201.1990
Epoch: 03, Loss: 2201.1733
Epoch: 04, Loss: 2201.1531
AP SCORE：  0.5983934064771681
AUC SCORE:  0.612563775510204
######################### 模块循环 ： 80 ##########################
Epoch: 00, Loss: 2201.1394
Epoch: 01, Loss: 2200.3926
Epoch: 02, Loss: 2198.9802
Epoch: 03, Loss: 2196.9773
Epoch: 04, Loss: 2194.4536
AP SCORE：  0.5988875211123637
AUC SCORE:  0.6116071428571428
Epoch: 00, Loss: 2191.4722
Epoch: 01, Loss: 2191.4553
Epoch: 02, Loss: 2191.4302
Epoch: 03, Loss: 2191.4048
Epoch: 04,

Epoch: 02, Loss: 2050.8618
Epoch: 03, Loss: 2050.8418
Epoch: 04, Loss: 2050.8274
AP SCORE：  0.5696752175322591
AUC SCORE:  0.5978954081632653
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 2050.8184
Epoch: 01, Loss: 2050.2153
Epoch: 02, Loss: 2049.0747
Epoch: 03, Loss: 2047.4580
Epoch: 04, Loss: 2045.4199
AP SCORE：  0.566239231249634
AUC SCORE:  0.5972576530612245
Epoch: 00, Loss: 2043.0120
Epoch: 01, Loss: 2042.9988
Epoch: 02, Loss: 2042.9788
Epoch: 03, Loss: 2042.9596
Epoch: 04, Loss: 2042.9453
AP SCORE：  0.5685324645602745
AUC SCORE:  0.596938775510204
######################### 模块循环 ： 98 ##########################
Epoch: 00, Loss: 2042.9364
Epoch: 01, Loss: 2042.3400
Epoch: 02, Loss: 2041.2109
Epoch: 03, Loss: 2039.6097
Epoch: 04, Loss: 2037.5918
AP SCORE：  0.5642963191340166
AUC SCORE:  0.5972576530612245
Epoch: 00, Loss: 2035.2070
Epoch: 01, Loss: 2035.1935
Epoch: 02, Loss: 2035.1737
Epoch: 03, Loss: 2035.1545
Epoch: 04, Loss: 2035.1404
AP SCORE：  